In [1]:
import pandas as pd
import glob
import matplotlib.pyplot as plt
import numpy as np
import glob
from IPython.display import display, HTML
import sys, os
import time


sys.path.append('/sps/lsst/users/rbonnetguerrini/ML4_transientV3')

from utils import rundir, datadir,decode_and_preprocess, pddir, All_visits_gen3,All_visits_UDEEP, embeddable_image

from PIL import Image
pd.set_option('display.max_columns', None)
from matplotlib import cm

import io
import base64
import tensorflow as tf

import warnings; warnings.simplefilter('ignore')
import pickle
import numpy as np
import pandas as pd

from Retrieve_data import retrieve_data_gen3, retrieve_data_gen3_all_col
from df_bogus import df_proba, df_association, df_proba_pytorch, retrieve_LC

from astropy import units as u
from astropy.time import Time
data_path ='/sps/lsst/groups/transients/HSC/fouchez/raphael'
from scipy.spatial import cKDTree

import pickle
import light_curve as lc
import umap

/pbs/software/centos-7-x86_64/anaconda/3.8/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:

from bokeh.io import output_notebook

output_notebook()

Loading BokehJS ...

In [3]:
with open("saved/full_dia_source_table_UDEEP.pkl", "rb") as pickle_file:
    full_dia_source_table_UDEEP = pickle.load(pickle_file)
    
with open("saved/pred_dia_obj_table.pkl", "rb") as pickle_file:
    clean_dia_obj_table = pickle.load(pickle_file)    


# LC creation

Producing the LC and the predictions for their individual sources

In [4]:
from Retrieve_data import retrieve_data_gen3_UDEEP


In [5]:
data = retrieve_data_gen3_UDEEP(visits=All_visits_UDEEP, rotate = False)
df_train, df_test, x_train, x_test, y_train, y_test = data.data(test_size=0.9)


In [6]:
df_train['y_test'] = df_train.pop('y_train')
df = pd.concat([df_train, df_test]).reset_index(drop=True)
x = np.concatenate((x_train, x_test), axis=0)

In [7]:
df

,diaSourceId,visit,band,apFlux,apFluxErr,snr,y_test
0,44279605288239168,103096,g,1372.913681,87.859638,16.781538,0
1,38556131869589582,89770,z,9668.880079,416.310188,10.539039,0
2,43738228955546173,101836,r,-961.765059,97.455413,6.276705,0
3,40136480118538293,93450,i,1669.136605,217.369661,11.026769,0
4,46679635160727606,108684,r,-656.1559,114.719909,5.556382,0
...,...,...,...,...,...,...,...
919378,47416537732087929,110400,y,5241.570817,927.110499,5.930499,0
919379,29338273058915697,68308,N921,-3902.933938,436.626296,5.179161,0
919380,23246289298784279,54124,y,4804.257233,824.73191,5.786623,0
919381,29338275206398539,68308,N921,3684.153918,436.002874,5.982116,0


In [8]:
len(x)

919383

In [9]:
sys.path.append('co_teaching/')
import torch
from model import CNN, CustomCNN

model_str='rotated_low_forget_factor_2'
save_dir =f'saved/{model_str}'
INPUT_SHAPE = (30, 30, 1)
NUM_CLASSES = 2
filters_1 = 32
filters_2 = 64
dropout_1 = 0.25
dropout_2 = 0.25
dropout_3 = 0.5
units = 128

model = CNN(input_channel=1, n_outputs=1)
model = CustomCNN(
    input_shape=INPUT_SHAPE,
    num_classes=NUM_CLASSES,
    filters_1=filters_1,
    dropout_1=dropout_1,
    filters_2=filters_2,
    dropout_2=dropout_2,
    units=units,
    dropout_3=dropout_3
)

model.load_state_dict(torch.load(f'co_teaching/cnn1_{model_str}.pth', map_location=torch.device('cpu')))


<All keys matched successfully>

In [10]:
df_tested = df_proba_pytorch(df, x, df['y_test'], model, model_str, batch_size= 64)

Shape of the test set: (919383, 30, 30, 1)
Test efficiency : 0.9204


In [11]:
df_tested

,diaSourceId,visit,band,apFlux,apFluxErr,snr,y_test,true_class,proba_transient_rotated_low_forget_factor_2,proba_transient_perc_rotated_low_forget_factor_2,proba_bogus_rotated_low_forget_factor_2,proba_bogus_perc_rotated_low_forget_factor_2,y_pred_rotated_low_forget_factor_2,bogus_transient_pred_rotated_low_forget_factor_2,false_positive,false_negative,true_positive,true_negative,class_proba_bogus_rotated_low_forget_factor_2,class_proba_transient_rotated_low_forget_factor_2
0,44279605288239168,103096,g,1372.913681,87.859638,16.781538,0,Real,4.451203e-07,0,1.000000,99,0,bogus,0,0,0,1,<10%,bogus
1,38556131869589582,89770,z,9668.880079,416.310188,10.539039,0,Real,2.350547e-31,0,1.000000,100,0,bogus,0,0,0,1,<10%,bogus
2,43738228955546173,101836,r,-961.765059,97.455413,6.276705,0,Real,1.942957e-38,0,1.000000,100,0,bogus,0,0,0,1,<10%,bogus
3,40136480118538293,93450,i,1669.136605,217.369661,11.026769,0,Real,7.916460e-09,0,1.000000,100,0,bogus,0,0,0,1,<10%,bogus
4,46679635160727606,108684,r,-656.1559,114.719909,5.556382,0,Real,1.212578e-30,0,1.000000,100,0,bogus,0,0,0,1,<10%,bogus
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
919378,47416537732087929,110400,y,5241.570817,927.110499,5.930499,0,Real,2.866076e-08,0,1.000000,100,0,bogus,0,0,0,1,<10%,bogus
919379,29338273058915697,68308,N921,-3902.933938,436.626296,5.179161,0,Real,2.164864e-12,0,1.000000,100,0,bogus,0,0,0,1,<10%,bogus
919380,23246289298784279,54124,y,4804.257233,824.73191,5.786623,0,Real,2.481512e-03,0,0.997518,99,0,bogus,0,0,0,1,<10%,bogus
919381,29338275206398539,68308,N921,3684.153918,436.002874,5.982116,0,Real,1.704016e-01,17,0.829598,82,0,bogus,0,0,0,1,>10%,bogus


In [12]:
dfs = {key: group for key, group in df_tested.groupby('band')}
band = ['g', 'r', 'i', 'z', 'y', 'N816', 'N921']
df_pred_g = dfs['g']
df_pred_r = dfs['r']
df_pred_i = dfs['i']
df_pred_z = dfs['z']
df_pred_y = dfs['y']
df_pred_N921 = dfs['N921']

In [13]:
df_asso_pred_g = df_association(df_pred_g, x)
df_asso_pred_r = df_association(df_pred_r, x)
df_asso_pred_i = df_association(df_pred_i, x)
df_asso_pred_z = df_association(df_pred_z, x)
df_asso_pred_y = df_association(df_pred_y, x)


In [14]:

pd.set_option('display.float_format', lambda x: '%.6f' % x)

def create_band_dia_obj_table(dia_source_id_band):
    with open("saved/full_dia_obj_table.pkl", "rb") as pickle_file:
        full_dia_obj_table = pickle.load(pickle_file)  
    full_dia_obj_table['consistency_band'] = None

    for objid, row_obj in full_dia_obj_table.iterrows(): 
        LC = dia_source_id_band[(dia_source_id_band['diaObjectId']==str(objid))]
        if len(LC)!=0: 
            LC_pred_transient = dia_source_id_band[(dia_source_id_band['diaObjectId']==str(objid)) & (dia_source_id_band['false_positive']==1)] 
            full_dia_obj_table.loc[objid, 'consistency_band']= len(LC_pred_transient)/len(LC)
        else : full_dia_obj_table = full_dia_obj_table.drop(objid)

    return full_dia_obj_table

In [ ]:
pred_dia_obj_table_g = create_band_dia_obj_table(df_asso_pred_g)
pred_dia_obj_table_r = create_band_dia_obj_table(df_asso_pred_r)
pred_dia_obj_table_i = create_band_dia_obj_table(df_asso_pred_i)
pred_dia_obj_table_z = create_band_dia_obj_table(df_asso_pred_z)
pred_dia_obj_table_y = create_band_dia_obj_table(df_asso_pred_y)


In [ ]:
with open("saved/pred_dia_obj_table_g.pkl", "wb") as pickle_file:
    pickle.dump(pred_dia_obj_table_g, pickle_file)
with open("saved/pred_dia_obj_table_r.pkl", "wb") as pickle_file:
    pickle.dump(pred_dia_obj_table_r, pickle_file)
with open("saved/pred_dia_obj_table_i.pkl", "wb") as pickle_file:
    pickle.dump(pred_dia_obj_table_i, pickle_file)
with open("saved/pred_dia_obj_table_z.pkl", "wb") as pickle_file:
    pickle.dump(pred_dia_obj_table_z, pickle_file)
with open("saved/pred_dia_obj_table_y.pkl", "wb") as pickle_file:
    pickle.dump(pred_dia_obj_table_y, pickle_file)

# LC plot

## Trying to get the best LC 

In [3]:
with open("saved/pred_dia_obj_table_g.pkl", "rb") as pickle_file:
    pred_dia_obj_table_g= pickle.load(pickle_file)
with open("saved/pred_dia_obj_table_r.pkl", "rb") as pickle_file:
    pred_dia_obj_table_r= pickle.load(pickle_file)
with open("saved/pred_dia_obj_table_i.pkl", "rb") as pickle_file:
    pred_dia_obj_table_i= pickle.load(pickle_file)
with open("saved/pred_dia_obj_table_z.pkl", "rb") as pickle_file:
    pred_dia_obj_table_z= pickle.load(pickle_file)
with open("saved/pred_dia_obj_table_y.pkl", "rb") as pickle_file:
    pred_dia_obj_table_y= pickle.load(pickle_file)
    

FileNotFoundError: [Errno 2] No such file or directory: 'saved/pred_dia_obj_table_g.pkl'

In [ ]:
def crossmatch_gaia(df):
    # Load the Gaia catalog from a pickle file
    with open("saved/source_cat_gaia.pkl", "rb") as pickle_file:
        gaia = pickle.load(pickle_file)
    
    # Stack Gaia catalog coordinates (RA, DEC) into an array and create a KDTree for fast spatial querying
    gaia_coords = np.vstack((gaia['ra'], gaia['dec'])).T
    gaia_tree = cKDTree(gaia_coords)
    
    # Define the matching tolerance (in degrees)
    tolerance = 0.0003
    
    # Stack the input DataFrame coordinates (RA, DEC) into an array
    df_coords = np.vstack((df['ra'], df['dec'])).T
    
    # Perform a spatial query to find Gaia catalog entries within the tolerance distance of each source in df
    matches = gaia_tree.query_ball_point(df_coords, r=tolerance)
    
    # Create a boolean array indicating whether each row has at least one match in Gaia
    in_gaia = np.array([len(match) > 0 for match in matches])
    
    # Add the boolean column to df
    df['in_gaia'] = in_gaia
    
    return df

In [ ]:
pd.set_option("display.max_rows", None)

removed_stars = crossmatch_gaia(pred_dia_obj_table_g)


In [ ]:
pd.set_option("display.max_rows", None)

try_finding_sn = removed_stars[removed_stars["in_gaia"]==0]
try_finding_sn = try_finding_sn[try_finding_sn['consistency']>0.8]

## Plot 

In [ ]:
try_finding_sn

In [6]:
from bokeh.io import output_notebook, push_notebook, show, curdoc, reset_output
from bokeh.plotting import figure, output_file, save
from bokeh.models import (ColumnDataSource, Slider, HoverTool, CategoricalColorMapper, Legend, Div,
                          Title, Tabs, Whisker, Circle, LegendItem, BoxZoomTool, ColorBar, LinearAxis, Range1d)
from bokeh.layouts import column, gridplot, row
from bokeh.palettes import Spectral10, Viridis, Spectral, Category10, brewer, tol, Turbo256
from bokeh.transform import factor_cmap, factor_mark

import pickle
from bokeh.transform import linear_cmap

from bokeh.transform import factor_cmap, factor_mark, linear_cmap
from astropy import units as u
from astropy.time import Time
data_path ='/sps/lsst/groups/transients/HSC/fouchez/raphael'

def plot_LC(df, model, model_name = None): 
    cutouts_array = []

    for idx, row in df.iterrows(): 
        visit = row['visit']
        features = pd.read_pickle(f'{data_path}/Detected_obj_sources_visit_{visit}_UDEEP_df.pkl')
        features = features.reset_index(drop=True)
        idx_img = features[features['diaSourceId'].astype(str)==row['diaSourceId']].index
        # Load coadd cutouts using pickle
        with open(f"{data_path}/Detected_obj_sources_visit_{visit}_UDEEP.pkl", 'rb') as f:
            diff = pickle.load(f)
        with open(f"{data_path}/Coadd_detected_obj_sources_visit_{visit}_UDEEP.pkl", 'rb') as f:
            coadd = pickle.load(f)
        # Load science cutouts using pickle
        with open(f"{data_path}/Science_detected_obj_sources_visit_{visit}_UDEEP.pkl", 'rb') as f:
            science = pickle.load(f)
            
        diff = np.array(diff)
        coadd = np.array(coadd)
        science = np.array(science)
    
        diff = diff.reshape(-1,30,30,1)
        diff = (diff- diff.min(axis=(1,2)).reshape((-1,1,1,1)) )/ (diff.max(axis=(1,2)).reshape((-1,1,1,1))- diff.min(axis=(1,2)).reshape((-1,1,1,1)))
        coadd = coadd.reshape(-1,30,30,1)
        coadd = (coadd- coadd.min(axis=(1,2)).reshape((-1,1,1,1)) )/ (coadd.max(axis=(1,2)).reshape((-1,1,1,1))- coadd.min(axis=(1,2)).reshape((-1,1,1,1)))
        science = science.reshape(-1,30,30,1)
        science = (science- science.min(axis=(1,2)).reshape((-1,1,1,1)) )/ (science.max(axis=(1,2)).reshape((-1,1,1,1))- science.min(axis=(1,2)).reshape((-1,1,1,1)))
        cutouts_array.append(diff[idx_img]) 
        df.at[idx, 'image'] = embeddable_image(diff[idx_img])
        df.at[idx, 'image_coadd'] = embeddable_image(coadd[idx_img])
        df.at[idx, 'image_science'] = embeddable_image(science[idx_img])
    

    cutouts_array = np.array(np.vstack((*cutouts_array,)))
    if model_name is None: 
        df = df_proba(df, cutouts_array, df['y_test'].values , model)

        model_name = model
        
    else : 
        y_test_tensor  = torch.tensor(df['y_test'].values, dtype=torch.long)
        df = df_proba_pytorch(df, cutouts_array, y_test_tensor , model, model_name)
    
    df['cal_mag'] = (np.array(df['apFlux'])* u.nJy).to(u.ABmag).value
    df['cal_mag_err'] = (np.array(df['apFluxErr'])* u.nJy).to(u.ABmag).value
    df['day_obs'] = Time(df['midpointMjdTai'], format='mjd').datetime64
    dataTP = ColumnDataSource(df.loc[df.true_positive == 1])
    dataTN = ColumnDataSource(df.loc[df.true_negative == 1])
    dataFP= ColumnDataSource(df.loc[df.false_positive == 1])
    dataFN = ColumnDataSource(df.loc[df.false_negative == 1])
    if model_name is None: 
        model_name = model
    # Prepare data for Bokeh
    GROUP2 = ['g', 'r', 'i' ,'z'] 

    colors = ['green','red','darkred','deeppink']
    # Create the plot
    p = figure(title="Light Curve", x_axis_label='Time', y_axis_label='Magnitude', x_axis_type='datetime')
    
    TP_glyph = p.x('day_obs', 'cal_mag', color='#FFC300',  source=dataTP, line_alpha=0.6, fill_alpha=0.6, size=8, legend_label = "True Positive")
    FP_glyph = p.star('day_obs', 'cal_mag', color='#FF5733',  source=dataFP, line_alpha=0.6, fill_alpha=0.6, size=8, legend_label = "False Positive")
    TN_glyph = p.circle('day_obs', 'cal_mag', color='#C70039',  source=dataTN, line_alpha=0.6, fill_alpha=0.6, size=8, legend_label = "True Negative")
    FN_glyph = p.square('day_obs', 'cal_mag', color='#b3b6b7',  source=dataFN, line_alpha=0.6, fill_alpha=0.6, size=8, legend_label = "False Negative")

    p.add_tools(HoverTool(tooltips=f"""
        <div>
            <div>
                <span style='font-size: 14px; color: #224499'> Real class : @true_class</span>
                <br>                
                <span style='font-size: 14px; color: #224499'> filter : @band</span>
                <br>
                <span style='font-size: 14px; color: #224499'> pred : @proba_transient_{model_name}</span>
                <img src='@image_coadd' height="50" width="50" style='float: left; margin: 5px 5px 5px 5px'></img>
                <img src='@image_science' height="50" width="50" style='float: left; margin: 5px 5px 5px 5px'></img>
                <img src='@image' height="50" width="50" style='float: left; margin: 5px 5px 5px 5px'></img>
            </div>
        </div>
        """))
    # Customize plot
    p.yaxis.axis_label = "Magnitude"
    p.xaxis.axis_label = "Time"
    p.y_range.flipped = True  # Magnitudes are usually plotted inversely
    curdoc().theme = 'dark_minimal'
    p.legend.location = "top_right"
    p.legend.title = "Classification"
    p.legend.click_policy = "hide"

    # Show plot
    output_notebook()
    show(p, notebook_handle =True)
    return p

In [ ]:
LC = retrieve_LC(3496112724792840206)
warnings.filterwarnings("ignore", message="out of range integer may result in loss of precision", category=UserWarning)
model_str='rotated_low_forget_factor_2'
save_dir =f'saved/{model_str}'
plot = plot_LC(LC, model, model_str)à